<a href="https://colab.research.google.com/github/Egaw0/ProgresBelajarku/blob/main/05_ReadVektorRaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vektor data di python

## Instalasi library

Pada pembelajaran ini library yang digunakan adalah Geopandas dan shapely.
Dalam instalasi library Geopandas sering terjadi error apabila hanya menggunakan `pip install geopandas` sehingga direkomendasikan menggunalan code instalasi yang langsung mengakses file di github.

In [ ]:
pip install geopandas

In [ ]:
pip install Shapely

In [ ]:
pip install pyproj

In [ ]:
#Library untuk data vektor
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoSeries, GeoDataFrame
import pyproj

In [ ]:
#Library dasar
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Setting Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("drive/MyDrive/04_Lecture/2023-2024 Genap/01_Geokomputasi")

In [ ]:
os.getcwd()

## Membuat data point

### Geoseries

In [ ]:
gs = GeoSeries([Point(106.814, -6.388), Point(106.832, -6.42), Point(106.911, -6.42)])
gs

In [ ]:
print(gs.crs)

In [ ]:
gs.crs ='epsg:4326'

In [ ]:
gs.plot()

### Geoseries dengan attribute

In [ ]:
data = {'name': ['a', 'b', 'c'],
        'lat': [-6.388, -6.42, -6.42],
        'lon': [106.814, 106.832, 106.91100]}

In [ ]:
print(data)

In [ ]:
geometry = gpd.points_from_xy(data['lon'], data['lat'])

In [ ]:
type(geometry)

In [ ]:
gs = GeoSeries(geometry, index=data['name'])
gs

In [ ]:
gs.crs = 'epsg:4326'

In [ ]:
gs.plot()

### GeoDataFrame

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
gdf = GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))

In [ ]:
gdf.plot(marker='*', color='red', markersize=50, figsize=(6, 6));

## Membaca file vektor geospasial

### Shapefile

In [ ]:
DsSukabumi = gpd.read_file("03_Raster/Data/ADMINISTRASIDESA_AR_25K.shp")

In [ ]:
DsSukabumi.head()

In [ ]:
DsSukabumi.crs

In [ ]:
DsSukabumi.plot(cmap='Set2')

In [ ]:
DsSukabumi[DsSukabumi['NAMOBJ'] == 'Babakan'].plot(cmap='Set2')

### OGC WFS GeoJSON

In [ ]:
pip install geojson

In [ ]:
import requests
import geojson

In [ ]:
wfs_url = "http://data.nanoos.org/geoserver/ows"
params = dict(service='WFS', version='1.0.0', request='GetFeature',
              typeName='oa:goaoninv', outputFormat='json')

r = requests.get(wfs_url, params=params)
wfs_geo = geojson.loads(r.content)

In [ ]:
print(type(wfs_geo))
print(wfs_geo.keys())
print(len(wfs_geo.__geo_interface__['features']))

In [ ]:
wfs_gdf = GeoDataFrame.from_features(wfs_geo)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [ ]:
wfs_gdf.plot(ax=world.plot(cmap='Set2', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

# Raster data di python

## Instalasi Library

Pada pembelajaran terkait data raster akan digunakan library turunan yakni rasterio.

In [ ]:
pip install pyproj

In [ ]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 43.5 MB/s eta 0:00:00


In [ ]:
pip install earthpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.4 MB/s eta 0:00:00


In [ ]:
import rasterio as rio
import rasterio.plot as rioplot
import pyproj

In [ ]:
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy as et

In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## Directory setting

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#os.chdir("drive/MyDrive/05_Classroom/01_GeoKomputasiKostumisasi_2022Genap")

In [ ]:
#os.getcwd()

## Satellite archives on the Cloud

Data raster akan menggunakan data Landsat berbasis cloud dari [Landsat AWS](https://docs.opendata.aws/landsat-pds/readme.html) dan [cloud google](https://cloud.google.com/storage/docs/public-datasets/landsat).

Cara menuliskan link data landsat pada cloud Amazon AWS atau google API adalah sebagai berikut:


**https://storage.googleapis.com/gcp-public-data-landsat/cx/LX/PPP/RRR/LXSS_LLLL_PPPRRR_YYYYMMDD_yyymmdd_CC_TX/**

**http://landsat-pds.s3.amazonaws.com/cx/LX/PPP/RRR/LXSS_LLLL_PPPRRR_YYYYMMDD_yyymmdd_CC_TX/**



Foldering: The **cx** refers to Collection (c1, c2), the **LX** refers to Landsat sensor (L9, L8, L7) , **PPP** refers to the scene’s path, **RRR** refers to the scene’s row. Kemudian detail penamaan file sebagai berikut:  

* L = Landsat
* X = Sensor
* SS = Satellite
* PPP = WRS path
* RRR = WRS row
* YYYYMMDD = Acquisition date
* yyyymmdd = Processing date
* CC = Collection number
* TX = Collection category

untuk data jakarta dengan **path : 122** dan **row : 064**. Bisa di check dl kondisi tampilan data pada [EarthExplorer](https://earthexplorer.usgs.gov/)

khusus untuk akses melalui aws diperlukan beberapa persyararan khusus
1. An AWS account.
2. AWS CLI installed and configured on your machine.
3. Python environment set up.
4. Installation of Boto3 and Rasterio libraries. Rasterio is useful for reading and writing raster datasets (like the ones provided by Landsat).




In [ ]:
print('Landsat on Google:')
#filepath = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/122/064/LC08_L1TP_122064_20210916_20210924_01_T1/LC08_L1TP_122064_20210916_20210924_01_T1_B4.TIF'
with rio.open(filepath) as src:
    print(src.profile)

In [ ]:
# membuka file dan membaca file yang kemudian disimpan dalam nama variable band4

with rio.open(filepath) as src:
    print(src.profile)
    band4 = src.read()

In [ ]:
ep.plot_bands(band4)

In [ ]:
type(band4)

numpy.ndarray

## Satellite Data as tif file

In [ ]:
worldview2 = "03_Raster/Data/10JAN25024728-M2AS-052647590010_01_P001.TIF"

In [ ]:
img = rio.open(worldview2)
rsWv = img.read()

In [ ]:
with rio.open(worldview2) as img:
  rsWv2 = img.read()
  print(img.profile)

In [ ]:
print('file rsWv2 jenis data: ', type(rsWv2))
print('file img jenis data: ', type(img))

In [ ]:
#prokjeksi
img.crs

In [ ]:
# dimensi citra
print('Lebar:', img.height)
print('Panjang:', img.width)

In [ ]:
#jumlah band
img.count

In [ ]:
#metadata
img.meta

In [ ]:
ep.plot_bands(rsWv2)